In [ ]:
import io
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import spacy
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.high_level import extract_text
from spacy.matcher import Matcher
import pandas as pd
import pdfkit
import docx2pdf
import doc2pdf
import win32com.client as win32
from win32com.client import constants
import os,os.path
import PyPDF2
import fitz
import glob

In [ ]:
#1 converting code

import os
from shutil import move
from django.shortcuts import render, redirect
from .models import Document
from django.contrib import messages
import fitz
from tqdm import tqdm
from docx2pdf import convert as docx2pdf_convert
import comtypes.client
import win32com.client
import comtypes

# I'm converting docx to pdf
def convert_docx_to_pdf(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    import comtypes

    # Initialize COM
    comtypes.CoInitialize()
    word = comtypes.client.CreateObject("Word.Application")
    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith(".docx"):
                input_file = os.path.join(root, filename)
                pdf_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.pdf")

                # Convert .docx to PDF using comtypes
                doc = word.Documents.Open(input_file)
                doc.SaveAs(pdf_file, FileFormat=17)  # FileFormat 17 corresponds to PDF
                doc.Close()
    word.Quit()

#I'm converting doc to pdf
def convert_doc_to_pdf(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        # Initialize COM
    comtypes.CoInitialize()

    word = win32com.client.DispatchEx("Word.Application")
    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith(".doc"):
                input_file = os.path.join(root, filename)
                pdf_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.pdf")

                # Convert .doc to PDF using pywin32 (Word Automation)
                doc = word.Documents.Open(input_file)
                doc.SaveAs(pdf_file, FileFormat=17)  # FileFormat 17 corresponds to PDF
                doc.Close()
    word.Quit()


def index(request):
    if request.method == "POST":
        documents = request.FILES.getlist("documents")
        unsupported_files = []
        for document in documents:
            # Check if it's a PDF file
            is_pdf = document.name.lower().endswith(".pdf")

            # Define the target directory based on file type
            if is_pdf:
                target_directory = r"C:\Users\alex\Downloads\Output"
            else:
                target_directory = r"C:\Users\alex\Downloads\input"

            if not document.name.lower().endswith((".doc", ".docx", ".pdf")):
                unsupported_files.append(document.name)
                continue

            # Move the uploaded file to the target directory
            file_path = os.path.join(target_directory, document.name)
            with open(file_path, 'wb') as destination:
                for chunk in document.chunks():
                    destination.write(chunk)

            # Perform the conversion if it's not a PDF
            if not is_pdf:
                # convert_doc_and_docx_to_pdf(target_directory, r"C:\Users\alex\Downloads\Output")
                convert_docx_to_pdf(target_directory, r"C:\Users\alex\Downloads\Output")
                convert_doc_to_pdf(target_directory, r"C:\Users\alex\Downloads\Output")
            # Create a Document object (optional)
            Document.objects.create(document=file_path)
        if unsupported_files:
            messages.error(request, "File format not supported for: " + ", ".join(unsupported_files))
        else:
            messages.success(request, "Documents Uploaded and Processed")
    return render(request, "index.html")

In [ ]:
pip install docx2pdf PyMuPDF

In [ ]:
import os
import comtypes.client

input_folder_path = r"C:\Users\alex\Downloads\Input"
output_folder_path = r"C:\Users\alex\Downloads\Output"
def convert_docx_to_pdf(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith(".docx"):
                input_file = os.path.join(root, filename)
                pdf_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.pdf")

                # Convert .docx to PDF using comtypes
                word = comtypes.client.CreateObject("Word.Application")
                doc = word.Documents.Open(input_file)
                doc.SaveAs(pdf_file, FileFormat=17)  # FileFormat 17 corresponds to PDF
                doc.Close()
                word.Quit()

def convert_doc_to_pdf(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith(".doc"):
                input_file = os.path.join(root, filename)
                pdf_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.pdf")

                # Convert .doc to PDF using comtypes
                word = comtypes.client.CreateObject("Word.Application")
                doc = word.Documents.Open(input_file)
                doc.SaveAs(pdf_file, FileFormat=17)  # FileFormat 17 corresponds to PDF
                doc.Close()
                word.Quit()

def convert_doc_and_docx_to_pdf(input_folder, output_folder):
    convert_docx_to_pdf(input_folder, output_folder)
    convert_doc_to_pdf(input_folder, output_folder)
    print("Done")

convert_doc_and_docx_to_pdf(input_folder_path, output_folder_path)

In [ ]:
#webdev

<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>BULK RESUME UPLOADER</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-rbsA2VBKQhggwzxH7pPCaAqO46MgnOM80zW1RWuH61DGLwZJEdK2Kadq2F9CUG65" crossorigin="anonymous">
  </head>
  <body>
    <div class="container mt-5 border rounded p-5 bg-light shadow" style="width:400px">
        {% if messages %}
            {% for message in messages %}
                <div class="text-center alert alert-success mb-3">{{message}}</div>
            {% endfor %}
        {% endif %}
        <h3 class="text-center">Upload Resume</h3>
        <form action="" method="post" class="mt-3" enctype="multipart/form-data">
            {% csrf_token %}
            <div class="mt-2">
                <label for="documents" style="font-size: 13px;">Documents</label>
                <br>
                <input required multiple type="file" name="documents" id="documents" class="form-control">
            </div>
            <div class="d-grid mt-5">
                <button class="btn btn-primary">Submit</button>
            </div>
        </form>
        <div class="progress mt-3" style="display: none;">
          <div class="progress-bar" role="progressbar" style="width: 0%;" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
        </div>
    </div>
  </body>
</html>


In [ ]:
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf_document:
        text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text += page.get_text()
    return text

# Function to extract name using spaCy
def extract_name(text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)

    nlp_text = nlp(text)

    # Define a pattern for extracting names (two consecutive proper nouns)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

    matcher.add('NAME', [pattern])

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        if 'name' not in span.text.lower():
            return span.text

# Function to extract mobile number using regex
def extract_mobile_number(text):
    phone = re.findall(re.compile(
        r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'),
        text)

    if phone:
        number = ''.join(phone[0])
        if len(number) >= 10:
            return '+' + number
        else:
            return number

# Function to extract email using regex
def extract_email(text):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", text)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

# Function to extract skills
def extract_skills(text):
    nlp = spacy.load('en_core_web_sm')

    # Load skills data from CSV
    skills_csv_path = r"C:\Users\alex\Downloads\skills.csv"
    skills_db_csv_path = r"C:\Users\alex\Downloads\skills_db.csv"

    if os.path.isfile(skills_csv_path) and os.path.isfile(skills_db_csv_path):
        data1 = pd.read_csv(skills_csv_path)
        data2 = pd.read_csv(skills_db_csv_path)
        data = pd.concat([data1, data2], ignore_index=True)
        skills = list(data.columns.values)
    else:
        print("CSV files not found. Please make sure the files exist at the specified paths.")
        return []

    # Process text to extract skills
    nlp_text = nlp(text)
    tokens = [token.text for token in nlp_text if not token.is_stop]

    skillset = []

    # Check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)

    # Check for bi-grams and tri-grams
    for chunk in nlp_text.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in skills:
            skillset.append(chunk_text)

    return [i.capitalize() for i in set([i.lower() for i in skillset])]

# Main function to process PDF files
def main():
    output_directory = r"C:\Users\alex\Downloads\Output"  # Directory containing PDF files

    # Create a list to store results for each PDF
    results = []

    # Iterate over PDF files in the output directory
    for filename in os.listdir(output_directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(output_directory, filename)

            # Extract text from the PDF file
            print(f"Extracting text from {filename}...")
            text = extract_text_from_pdf(pdf_path)

            # Extract name, email, mobile number, and skills
            name = extract_name(text)
            email = extract_email(text)
            mobile_number = extract_mobile_number(text)
            skills = extract_skills(text)

            # Store the results in a dictionary
            result = {
                "Filename": filename,
                "Name": name,
                "Email": email,
                "Mobile Number": mobile_number,
                "Skills": skills
            }

            # Append the result to the list
            results.append(result)

    # Print the results
    for result in results:
        print(f"Resume Name: {result['Filename']}")
        print(f"Name: {result['Name']}")
        print(f"Email: {result['Email']}")
        print(f"Mobile Number: {result['Mobile Number']}")
        print(f"Skills: {', '.join(result['Skills'])}")
        print("\n")

if __name__ == "__main__":
    main()

In [ ]:
import re

In [ ]:
 lst = "asdfghhjaklqwertzxvcb"
match = re.findall(r"a", lst)

print(match)

In [ ]:
pdf_path = r"C:\Users\alex\Downloads\testcv\testres.pdf"
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf_document:
        text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text += page.get_text()
    return text
extract_text_from_pdf(pdf_path)

In [ ]:
pdf_path = r"C:\Users\alex\Downloads\Output\resume1.pdf"
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf_document:
        text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text += page.get_text()
    return text
extract_text_from_pdf(pdf_path)

In [ ]:
pdf_path = r"C:\Users\alex\Downloads\Output\testdoc_resume4.pdf"
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf_document:
        text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text += page.get_text()
    return text
extract_text_from_pdf(pdf_path)

In [ ]:
pdf_path = r"C:\Users\alex\Downloads\Output\testdoc2-resume.pdf"
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf_document:
        text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text += page.get_text()
    return text
extract_text_from_pdf(pdf_path)

In [ ]:
pdf_path = r"C:\Users\alex\Downloads\Output\testdoc5_resume.pdf"
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf_document:
        text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text += page.get_text()
    return text
extract_text_from_pdf(pdf_path)

In [ ]:
import re

def extract_experience_and_domain(text):
    # Regular expression patterns to match years of experience and domain
    experience_pattern = r'(\d+) years of experience'
    domain_pattern = r'(Office Administrator|DATA ENGINEER|FULL STACK PYTHON DEVELOPER|Amazon Associate|SOFTWARE ENGINEER|Warehouse Associate|Laboratory Inventory Assistant)'

    # Search for years of experience and domain in the resume text
    experience_match = re.search(experience_pattern, resume1_text)
    domain_match = re.search(domain_pattern, resume1_text)

    # Initialize variables to store the results
    years_of_experience = None
    domain = None

    # Extract years of experience if found
    if experience_match:
        years_of_experience = int(experience_match.group(1))

    # Extract domain if found
    if domain_match:
        domain = domain_match.group(1)

    return years_of_experience, domain

# Example usage:
# resume1_text = "Jason Miller\nAmazon Associate\nProfile\nExperienced Amazon Associate with five years’ tenure in a shipping yard..."
# resume2_text = "Omkar Pathak\nSOFTWARE ENGINEER · FULL STACK PYTHON DEVELOPER\nPune, Maharashtra, India..."
# resume3_text = "Highly organized and detail-oriented office manager with over 5 years of experience managing administrative tasks..."

years_exp1, domain1 = extract_experience_and_domain(resume1_text)
# years_exp2, domain2 = extract_experience_and_domain(resume2_text)
# years_exp3, domain3 = extract_experience_and_domain(resume3_text)

print(f"Resume 1: {years_exp1} years of experience in {domain1}")
# print(f"Resume 2: {years_exp2} years of experience in {domain2}")
# print(f"Resume 3: {years_exp3} years of experience in {domain3}")


In [ ]:
import re
from datetime import datetime

resume_text1 = "NOVEMBER 3, 2019 \nOMKAR PATHAK · RÉSUMÉ \n1 \n \nOmkar Pathak \nSOFTWARE ENGINEER · FULL STACK PYTHON DEVELOPER \nPune, Maharashtra, India \n(+91) 8087996634 \n| \nomkarpathak27@gmail.com \n| \nwww.omkarpathak.in \n|     \n OmkarPathak \n|     \n omkar-pathak-94473811b \n \n“Make the change that you want to see in the world.” \n \nExperience   \n \nSchlumberger \nPune, Maharashtra, India \nDATA ENGINEER \nJuly 2018 - Present \n• Responsible for implementing and managing an end-to-end CI/CD Pipeline with custom validations for Informatica migrations which \nbrought migration time to 1.5 hours from 9 hours without any manual intervention \n• Enhancing, auditing and maintaining custom data ingestion framework that ingest around 1TB of data each day to over 70 business \nunits \n• Working with L3 developer team to ensure the discussed Scrum PBI’s are delivered on time for data ingestions \n• Planning and Executing QA and Production Release Cycle activities \nTruso \nPune, Maharashtra, India \nFULL STACK DEVELOPER INTERN \nJune 2018 - July 2018 \n• Created RESTful apis \n• Tried my hands on Angular 5/6 \n• Was responsible for Django backend development \nPropeluss \nPune, Maharashtra, India \nDATA ENGINEERING INTERN \nOctober 2017 - January 2018 \n• Wrote various automation scripts to scrape data from various websites. \n• Applied Natural Language Processing to articles scraped from the internet to extract different entities in these articles using entity \nextraction algorithms and applying Machine Learning to classify these articles. \n• Also applied KNN with LSA for extracting relevant tags for various startups based on their works. \nGeeksForGeeks \nPune, Maharashtra, India \nTECHNICAL CONTENT WRITER \nJuly 2017 - September 2017 \n• Published 4 articles for the topics such as Data Structures and Algorithms and Python \nSofttestlab Technologies \nPune, Maharashtra, India \nWEB DEVELOPER INTERN \nJune 2017 - July 2017 \n• Was responsible for creating an internal project for the company using PHP and Laravel for testing purposes \n• Worked on a live project for creating closure reports using PHP and Excel \n \nProjects   \n \nPyresparser \nAPI/Python Package \nPERSONAL PROJECT \nJuly 2019 - Present \n• A simple resume parser used for extracting information from resumes \n• Extract information from thousands of resumes in just a few seconds \n• Author and maintainer of this project \nGarbage Level Monitoring System \nIoT \nTEAM PROJECT \nOctober 2017 - May 2018 \n• To find a economical and smarter alternative to current garbage problems \n• Users can monitor levels of all garbage bins from a global dashboard provided \n• Was responsible for Django backend development \nNOVEMBER 3, 2019 \nOMKAR PATHAK · RÉSUMÉ \n2 \n \nPygorithm \nAPI / Python Package \nPERSONAL PROJECT \nJuly 2017 - Present \n• Author and maintainer of this project \n• An educational library to teach all the major algorithms \n• Got covered in Fosstack, FullStackFeed, Kleiber and Tagged under Hotest Github Project on ITCodeMonkey \n \n \n \nSmart Surveillance System using Raspberry Pi and Face Recognition \nIoT \nPERSONAL PROJECT \nJanuary 2017 - February 2017 \n• Face Recognition using OpenCV and Python \n• Raspberry Pi was used as the data server \n• User notified if any suspicious activity detected in real time \n \nPassword Strength Evaluator using Machine Learning \nMachine Learning \nPERSONAL PROJECT \nMarch 2017 \n• SVM algorithm used for training and classification \n• Flask framework used \n• Self-generated dataset \n \nEducation   \n \nMarathwada Mitra Mandal’s College of Engineering \nPune, Maharashtra, India \nB.E. IN COMPUTER ENGINEERING \n2014 - 2018 \n• Aggregate 74% \nSkills   \n \n \nProgramming Languages: Python, C, PHP, C++, Shell Script \nFrontend Technologies: HTML, CSS, JavaScript, Angular 6/7 \nBackend Technologies: Django, Flask (Python), Laravel (PHP) \nOperating Systems:  Linux, Unix, Windows \nDatabases: MySQL, SQLite, MongoDB \nOther:  Git, NLP, Scikit-Learn, OpenCV, Cloud (GCP, Azure, DigitalOcean) \n \nHonors & Awards   \n \n \n2018 Top rated Python developer, in Pune and Fifth in India at Github \nIndia \n2018 Quora Top Writer, \nIndia \n2018 Awarded ‘The Best Outgoing Student Award 2017-18’, \nMMCOE, Pune \n2018 Won 2nd Prize, in an Hackathon organized by MIT-ADT Persona Fest 2018 \nPune \n \n2018 \nBest Paper Award, in National Level Conference on “Emerging Trends in Computing , Analytics \nand Security - 2018”(NCETCAS-2018) \nMMCOE, Pune \n \nExtracurricular Activities   \n \nContributor in Pune PyCon 2018 \nPUNE, MAHARASHTRA, INDIA \n2018 \n• Was a part of Website Designing and volunteering committee \nNOVEMBER 3, 2019 \nOMKAR PATHAK · RÉSUMÉ \n3 \n \nMentor at GirlScript Summer of Code 2019 \nPUNE, MAHARASHTRA, INDIA \n2019 \n• Mentored 4+ teams in various domains \nOrganizing head for the National level technical event - \nInnovatus \nPUNE, MAHARASHTRA, INDIA \n2018 \n• Organized project competitions \n \nWorkshop on IoT and Python \nMMCOE, PUNE \n10 Jan 2017 \n• Conducted a workshop for second year students to give them a brief overview about IoT by completing three mini projects and taught \nthem basics of Python programming language \n \nPublications   \n \n \nSmart Surveillance System using Raspberry Pi and Face \nRecognition \n \nGarbage Level Monitoring System \n \nDOI10.17148/IJARCCE.2017.64117 \n \nInterests   \n \n• Competitive Programming \n• Photography \n• Sketching \n• Reading/Writing on Quora \n• Contributing to Open Source projects \n"

def calculate_experience(date_ranges):
    today = datetime.now()
    total_experience = 0

    for start_date, end_date in date_ranges:
        if end_date.lower() == 'present':
            end_date = today
        else:
            end_date = datetime.strptime(end_date, "%B %Y")

        start_date = datetime.strptime(start_date, "%B %Y")
        experience = (end_date - start_date).days / 365  # Calculate experience in years
        total_experience += experience

    return total_experience

def extract_experience_from_resume(resume_text):
    # Split the text into lines
    lines = resume_text.split('\n')

    # Initialize variables to store sections and date ranges
    current_section = ''
    date_ranges = []

    # Define a function to check if a line is a section header
    def is_section_header(line):
        return re.match(r'(Experience|Employment History)', line, re.IGNORECASE)

    for line in lines:
        if is_section_header(line):
            # Start a new section
            if current_section:
                # Extract date ranges from the current section
                experience_dates = re.findall(r'(\w+ \d{4}) — (\w+ \d{4}|\w+ present)', current_section)
                date_ranges.extend(experience_dates)
            current_section = line
        else:
            # Add the line to the current section
            current_section += '\n' + line

    # Extract date ranges from the last section
    if current_section:
        experience_dates = re.findall(r'(\w+ \d{4}) — (\w+ \d{4}|\w+ present)', current_section)
        date_ranges.extend(experience_dates)

    # Calculate total experience in years
    total_experience = calculate_experience(date_ranges)

    return total_experience

# Example usage:

experience = extract_experience_from_resume(resume_text)
print(f"Total Experience: {experience:.2f} years")

In [ ]:
import docx2txt
from pdfminer.high_level import extract_text
import sklearn

In [ ]:
job_discription = docx2txt.process(r'C:\Users\alex\Downloads\testcv\JD.docx')
resume = docx2txt.process(r'C:\Users\alex\Downloads\testcv\testdoc_resume4.docx')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\alex\\Downloads\\testcv\\JD.docx'

In [ ]:
print(resume)

In [ ]:
content = [job_discription, resume]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
matrix = cv.fit_transform(content)

##
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(content)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix1 = cosine_similarity(matrix)

##
similarity_matrix = cosine_similarity(tfidf_matrix)


In [ ]:
!pip uninstall nvidia-cublas-cu11
#nvidia-cuda-cupti-cu11
# nvidia-cuda-nvrtc-cu11    11.7.99
# nvidia-cuda-runtime-cu11  11.7.99
# nvidia-cudnn-cu11         8.5.0.96
# nvidia-cufft-cu11         10.9.0.58
# nvidia-curand-cu11        10.2.10.91
# nvidia-cusolver-cu11      11.4.0.1
# nvidia-cusparse-cu11      11.7.4.91
# nvidia-nccl-cu11          2.14.3
# nvidia-nvtx-cu11          11.7.91

In [ ]:
print(similarity_matrix1)
print(similarity_matrix)

In [ ]:
!pip install -U spacy
!pip install spacy_transformers

  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached nvidia_nccl_cu11-2.14.3-py3-none-manylinux1_x86_64.whl (177.1 MB)
  Using cached nvidia_nvtx_cu11-11.7.91-py3-none-manylinux1_x86_64.whl (98 kB)


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

'3.6.1'

In [ ]:
cv_data = json.load(open(r'/home/rushikeshmallanath/Downloads/spacyResume/dataset.json', 'r'))

In [ ]:
len(cv_data)

1014

In [ ]:
cv_data[0]

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [ ]:
!python -m spacy init fill-config /home/rushikeshmallanath/Downloads/spacyResume/base_config.cfg  /home/rushikeshmallanath/Downloads/spacyResume/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/home/rushikeshmallanath/Downloads/spacyResume/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = []

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue

            entity_indices = entity_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + "    " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
    return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size = 0.3)

In [ ]:
len(train), len(test)

(709, 305)

In [ ]:
file = open(r'/home/rushikeshmallanath/Downloads/spacyResume/train_file.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk(r'/home/rushikeshmallanath/Downloads/spacyResume/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk(r'/home/rushikeshmallanath/Downloads/spacyResume/test_data.spacy')

file.close()

100%|████████████████████████████████| 305/305 [00:01<00:00, 154.45it/s]


In [ ]:
!python -m spacy train /home/rushikeshmallanath/Downloads/spacyResume/config.cfg  --output /home/rushikeshmallanath/Downloads/spacyResume/output --paths.train /home/rushikeshmallanath/Downloads/spacyResume/train_data.spacy --paths.dev /home/rushikeshmallanath/Downloads/spacyResume/test_data.spacy --gpu-id -1

ℹ Saving to output directory:
/home/rushikeshmallanath/Downloads/spacyResume/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    137.15    0.00    0.00    0.00    0.00
  0     200       4349.42  14445.46   46.59   51.32   42.66    0.47
  0     400       2056.43   9226.96   53.54   62.13   47.04    0.54
  0     600        854.38   7123.34   56.18   53.50   59.13    0.56
  1     800        865.67   4723.51   72.05   72.16   71.93    0.72
  1    1000        512.00   4349.79   73.42   75.67   71.30    0.73
  1    1200        548.29   4256.58   75.17   81.21   69.97    0.75
  1    1400        333.86   3594.59   69.71   77.86 

In [ ]:
nlp = spacy.load('/home/rushikeshmallanath/Downloads/spacyResume/output/model-best')

In [ ]:
!pip install PyMuPDF

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/3a/d2/a6042d0dc90a609e487169031f0af5d789acb01d8a8f73ba31fffcfe225d/PyMuPDF-1.23.3-cp38-none-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.23.3 from https://files.pythonhosted.org/packages/34/cf/bb6a3bad0ffdea1fb9fcae53de6c1b9187746c32e17377ff8f36b9d98ee4/PyMuPDFb-1.23.3-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 2.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:00:01


In [ ]:
import sys, fitz

fname = '/home/rushikeshmallanath/Downloads/BCBS1.pdf'
doc = fitz.open(fname)

In [ ]:
text = " "
for page in doc:
    text = text + str(page.get_text())

In [ ]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, " - ", ent.label_)

NameError: name 'nlp' is not defined